## データの取得・確認

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path('__file__').resolve().parent.parent.parent.parent))

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from termcolor import cprint

from src.modules.scripts.evaluation import score

cancer = datasets.load_breast_cancer()
cancer_data = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
cancer_target = pd.Series(data=cancer.target, name='target')

X_train, X_test, y_train, y_test = model_selection.train_test_split(cancer_data, cancer_target)

cprint('Cancer Data (samples: {}, features: {})'.format(*cancer_data.shape), 'blue', attrs=['bold'])
cprint('Cancer Target (samples: {})'.format(*cancer_target.shape), 'blue', attrs=['bold'])


Cancer Data (samples: 569, features: 30)
Cancer Target (samples: 569)


In [2]:
cancer_data.head(3)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [3]:
cancer_target.unique()


array([0, 1])

## データ前処理

In [4]:
def standardize(X):
    """標準化する.
    
    X: データセット
    """
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)


def add_ones(X):
    """特徴量の0列目に1を加える.
    
    X: データセット
    """
    X['0'] = 1
    return X.sort_index(axis=1, ascending=True)


X_train_std = standardize(X_train)
X_test_std = standardize(X_test)

# バイアス項を追加
X_train_std = add_ones(X_train_std)
X_test_std = add_ones(X_test_std)

X_train_std.head(3)


,0,area error,compactness error,concave points error,concavity error,fractal dimension error,mean area,mean compactness,mean concave points,mean concavity,...,worst area,worst compactness,worst concave points,worst concavity,worst fractal dimension,worst perimeter,worst radius,worst smoothness,worst symmetry,worst texture
294,1,-0.643446,-0.780373,-0.638357,-0.821379,-0.902488,-0.505890,-0.408827,-0.807764,-0.980388,...,-0.606436,-0.702144,-0.822136,-1.078232,-0.823321,-0.594776,-0.611714,-0.089617,-0.828089,-1.309856
312,1,-0.369575,0.080366,-0.219110,-0.054112,0.175860,-0.471368,-0.490883,-0.647454,-0.640082,...,-0.493129,-0.241711,-0.506086,-0.488786,-0.098458,-0.491153,-0.469344,-0.557477,-0.526273,-1.485708
543,1,-0.539700,-0.676304,-0.462279,-0.561066,-0.920101,-0.372692,-0.688371,-0.460682,-0.771205,...,-0.474332,-0.759072,-0.575317,-0.823328,-1.084184,-0.478127,-0.432204,-1.106314,-0.667120,1.896182


## モデルの構築・訓練

In [5]:
class LogisticRegression(object):
    """勾配降下法に基づくロジスティック回帰."""

    def __init__(self, alpha=0.01, eps=1e-6):
        """
        ---パラメータ-------
        alpha: 学習率
        eps: 収束の目安
        --------------------
        
        ---属性-------------
        _w: パラメータ（重み）
        --------------------
        """
        self.alpha = alpha
        self.eps = eps

        self._w = 0

    def fit(self, X, y):
        """訓練データで学習する.
        
        ---パラメータ-------
        X: 訓練データセット
        y: ターゲット
        --------------------
        
        """
        # パラメータの初期値はランダムに設定する
        theta = np.random.rand(X.shape[1])
        error = self.J(X, y, theta)
        count = 0
        diff = 1

        # コスト関数の値が殆ど変動しなくなったら収束とする
        while diff > self.eps:
            # パラメータの更新処理
            grad = (1 / len(y)) * np.dot(self.h(X, theta) - y, X)
            theta = theta - self.alpha * grad

            # 収束状況を確認するための処理
            current_error = self.J(X, y, theta)
            diff = error - current_error
            error = current_error
            count += 1
            print('\r【{}回目】Error: {}, Diff: {}, Grad: {}'.format(count, error, diff, grad.sum()), end='')

        self._w = theta
        return self

    def predict(self, X):
        """クラスラベルを予測する.
        
        X: テストデータセット
        """
        return np.where(self.h(X, self._w) >= 0.5, 1, 0)

    def activate(self, z):
        """ロジスティック関数（活性化関数）.
        
        z: 仮説関数の予測値
        """
        return 1 / (1 + np.exp(-z))

    def h(self, X, theta):
        """仮説関数.
        
        X: 訓練データセット
        theta: パラメータ（重み）
        """
        return self.activate(np.dot(X, theta))

    def J(self, X, y, theta):
        """目的関数.
        
        X: 訓練データセット
        y: ターゲット
        theta: パラメータ（重み）
        """
        delta = 1e-7  # np.log()に0が混在しないようにする
        return - (1 / len(y)) * (
            np.sum(y * np.log(self.h(X, theta) + delta) + (1 - y) * np.log(1 - self.h(X, theta) + delta)))


In [6]:
lr = LogisticRegression()
lr.fit(X_train_std, y_train)


【105回目】Error: 0.6492472647088537, Diff: 0.012881955090258579, Grad: 5.980335913896307

【216回目】Error: 0.2327419141738341, Diff: 0.0009204312576412688, Grad: 1.5358934502090544

【327回目】Error: 0.17548860056675128, Diff: 0.0003007166756125168, Grad: 0.8049152775125269

【440回目】Error: 0.15073006071809222, Diff: 0.00016398988418800386, Grad: 0.5368697163882423

【550回目】Error: 0.13597411440329646, Diff: 0.00011101005501512606, Grad: 0.40877011415680375

...

【1335回目】Error: 0.09428716272008557, Diff: 2.6774551576291827e-05, Grad: 0.16772535956348644

【1448回目】Error: 0.09145281597094852, Diff: 2.353384254875912e-05, Grad: 0.15578425331420348

【1559回目】Error: 0.08899080560391746, Diff: 2.093367164682225e-05, Grad: 0.1457099428005555

【1668回目】Error: 0.0868291708416903, Diff: 1.881132310110889e-05, Grad: 0.1370839698446151

【12252回目】Error: 0.04777824989069925, Diff: 9.999895328668962e-07, Grad: 0.02234918931255615738

## 予測・評価

In [7]:
predict = pd.DataFrame(lr.predict(X_train_std), columns=['TARGET'])
predict.head(10)

,TARGET
0,1
1,1
2,1
3,0
4,0
5,0
6,0
7,0
8,1
9,0


In [8]:
train_score = score(predict, y_train)
test_score = score(pd.DataFrame(lr.predict(X_test_std), columns=['target']), y_test)
cprint('train_score: {}\ntest_score: {}'.format(train_score, test_score), 'red', attrs=['bold'])

train_score: 0.9906103286384976
test_score: 0.972027972027972
